# 4. Loan Trajectories

Generate and analyze individual loan paths using the Loan Trajectory Model.

## Contents
1. Model Overview
2. Trajectory Generation
3. Payment Analysis
4. Default Timing

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

from privatecredit.models import LoanTrajectoryModel
from privatecredit.models.loan_trajectory import LoanTrajectoryConfig

## 1. Model Overview

The Loan Trajectory Model is an autoregressive transformer with:
- State classification head
- Payment regression head
- Diffusion-based generation for continuous outputs

In [ ]:
config = LoanTrajectoryConfig(
    n_states=7,
    hidden_dim=128,
    n_heads=4,
    n_layers=4,
    max_seq_len=60,
    diffusion_steps=100
)

model = LoanTrajectoryModel(config)
print(f"Parameters: {sum(p.numel() for p in model.parameters()):,}")

## 2. Trajectory Generation

In [ ]:
# Generate sample trajectories (using untrained model for demo)
n_loans = 100
n_periods = 60

# Simulate simple loan paths
np.random.seed(42)

# State sequences (0=Performing, 4=Default, 5=Prepaid)
states = np.zeros((n_loans, n_periods), dtype=int)

for i in range(n_loans):
    for t in range(1, n_periods):
        if states[i, t-1] >= 4:  # Absorbing
            states[i, t] = states[i, t-1]
        else:
            r = np.random.rand()
            if r < 0.002:  # Default
                states[i, t] = 4
            elif r < 0.015:  # Prepay
                states[i, t] = 5
            elif r < 0.03:  # Worsen
                states[i, t] = min(3, states[i, t-1] + 1)
            elif r < 0.05:  # Improve
                states[i, t] = max(0, states[i, t-1] - 1)
            else:
                states[i, t] = states[i, t-1]

print(f"State trajectories shape: {states.shape}")

In [ ]:
# Visualize sample trajectories
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
state_names = ['Perf', '30D', '60D', '90D', 'Def', 'Pre', 'Mat']

for idx, ax in enumerate(axes.flat):
    ax.plot(states[idx], marker='o', markersize=3)
    ax.set_yticks(range(7))
    ax.set_yticklabels(state_names)
    ax.set_xlabel('Month')
    ax.set_title(f'Loan {idx+1} Trajectory')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Payment Analysis

In [ ]:
# Simulate payments (fraction of scheduled)
initial_balance = 100000
monthly_payment = 1500

payment_fractions = np.ones((n_loans, n_periods))
balances = np.zeros((n_loans, n_periods))

for i in range(n_loans):
    balances[i, 0] = initial_balance
    for t in range(n_periods):
        state = states[i, t]
        if state == 4:  # Default
            payment_fractions[i, t] = 0
        elif state == 5:  # Prepaid
            payment_fractions[i, t] = balances[i, t-1] / monthly_payment if t > 0 else 1
        elif state >= 1:  # Delinquent
            payment_fractions[i, t] = np.random.uniform(0, 0.5)
        else:
            payment_fractions[i, t] = np.random.uniform(0.95, 1.0)
        
        if t > 0:
            principal_paid = monthly_payment * payment_fractions[i, t] * 0.3
            balances[i, t] = max(0, balances[i, t-1] - principal_paid)

# Plot payment distribution over time
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(payment_fractions.mean(axis=0), label='Mean', linewidth=2)
axes[0].fill_between(range(n_periods),
                     np.percentile(payment_fractions, 25, axis=0),
                     np.percentile(payment_fractions, 75, axis=0),
                     alpha=0.3, label='25-75 percentile')
axes[0].set_xlabel('Month')
axes[0].set_ylabel('Payment Fraction')
axes[0].set_title('Payment Performance Over Time')
axes[0].legend()

axes[1].plot(balances.mean(axis=0), label='Mean', linewidth=2)
axes[1].fill_between(range(n_periods),
                     np.percentile(balances, 25, axis=0),
                     np.percentile(balances, 75, axis=0),
                     alpha=0.3)
axes[1].set_xlabel('Month')
axes[1].set_ylabel('Balance ($)')
axes[1].set_title('Balance Amortization')

plt.tight_layout()
plt.show()

## 4. Default Timing

In [ ]:
# Analyze default timing
default_times = []
for i in range(n_loans):
    default_idx = np.where(states[i] == 4)[0]
    if len(default_idx) > 0:
        default_times.append(default_idx[0])

print(f"Number of defaults: {len(default_times)} / {n_loans}")

if default_times:
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.hist(default_times, bins=20, edgecolor='white', color='steelblue')
    ax.set_xlabel('Month')
    ax.set_ylabel('Number of Defaults')
    ax.set_title('Default Timing Distribution')
    ax.axvline(np.mean(default_times), color='red', linestyle='--', label=f'Mean: {np.mean(default_times):.1f}')
    ax.legend()
    plt.tight_layout()
    plt.show()

## Summary

- Loan trajectories capture state, payment, and balance evolution
- Default timing varies by loan characteristics
- Autoregressive model enables path-dependent simulation

**Next:** Portfolio simulation (Notebook 05)